In [0]:
import numpy as np
import re
import nltk
import contractions
import string
import pandas as pd
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim import utils 
from gensim.test.utils import get_tmpfile
from nltk.corpus import stopwords
import nltk
import en_core_web_sm
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
nlp = en_core_web_sm.load()
nltk.download('stopwords')
nltk.download('punkt')
porter=PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
pip install contractions

     |████████████████████████████████| 245kB 4.1MB/s 
     |████████████████████████████████| 317kB 35.6MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81692 sha256=40ce8eeaf8aa36043175b66b8a86c93e914b37460e9daf7b69b5ab711a1b98a5
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [0]:
REGEX = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
             '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

NEWLINE_REGEX = ('[\n\r\t]')
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)


def replace_ne(text: str, nlp) -> str:
    '''
    This function extracts the name entities, and then replace them with the ne labels.
    :param text:
    :type text:
    :return:
    :rtype:
    '''
    doc = nlp(text)
    for ent in doc.ents:
        text = text.replace(ent.text,ent.label_ )
    return text
def textClean(text):
    """
    Get rid of the non-letter and non-number characters
    """
    text = re.sub(REGEX, " ", text)
    text = re.sub(NEWLINE_REGEX, " ", text)
    text=re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)
    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return (text)

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

def cleanup(text):
    text = textClean(text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = replace_ne(text,nlp)
    text= replace_contractions(text)

    text= stemSentence(text)
    return text

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import pandas as pd 
import numpy as np

#group 4 
#have to read seperately perez_rosas_etal_2018= pd.read_csv()

#misinformation
misinfo_asr_ft_etal_emergent= pd.read_csv("/content/drive/My Drive/Colab Notebooks/emergent/emergent_phase2_clean_2018_7_2.csv", header= 0)

misinfo_asr_ft_etal_emergent.head(1)

,emergent_url_phase1,category_phase1,fact_tag_phase1,claim_phase1,claim_description_phase1,article_tags_phase1,article_date_phase1,article_tracking_body_phase1,original_url_phase1,error_phase2,original_article_text_phase2,article_title_phase2,publish_date_phase2,author_phase2
0,http://www.emergent.info/disney-collector-is-f...,Culture,Unverified,Claim: The woman behind the hugely popular DC ...,The channel has racked up more than 2.8 billio...,Daiane DeJesus&DC Toys Collector&Porm&Sandy Su...,Jul 3,The Daily Mail published a story on Feb 24 tha...,http://www.dailymail.co.uk/news/article-295824...,No Error,"The highest paid 'performer' on You Tube, who ...",Brazilian EX-porn star Diane DeJesus is figure...,2015-02-24,Paul Thompson In Orlando


In [0]:
#droping unverified data
misinfo_emergent=misinfo_asr_ft_etal_emergent[misinfo_asr_ft_etal_emergent.fact_tag_phase1 != 'Unverified']
misinfo_emergent=misinfo_emergent.reset_index(drop=True)

#dropping other columns only taking misinfo_asr_ft_etal_emergent['original_article_text_phase2'] and label
misinfo_emergent=misinfo_emergent[['fact_tag_phase1','original_article_text_phase2']]

#renaming columns
misinfo_emergent= misinfo_emergent.rename(columns={'fact_tag_phase1':'Label','original_article_text_phase2':'Article'})

#converting label with True and False 
misinfo_emergent=misinfo_emergent.replace('true','True')
misinfo_emergent=misinfo_emergent.replace('false','False')

misinfo_emergent.head(2)

,Label,Article
0,False,You know youre in trouble when Tom Brokaw is o...
1,True,DUBAI - A prominent Saudi Arabian cleric has ...


In [0]:
misinfo_check=misinfo_asr_ft_etal_emergent[misinfo_asr_ft_etal_emergent.fact_tag_phase1 == 'Unverified']
misinfo_check.shape

(40, 14)

In [0]:
misinfo_snopes_original= pd.read_csv("/content/drive/My Drive/Colab Notebooks/snopes/snopes_checked_v02.csv",header= 0, encoding="ISO-8859-1")
misinfo_snopes=misinfo_snopes_original[['fact_rating_phase1','original_article_text_phase2']]

#resetting index
misinfo_snopes=misinfo_snopes.reset_index(drop=True)

#dropping other labels in the data set 
#misinfo_snopes=misinfo_snopes[(misinfo_snopes['fact_rating_phase1'] == 'true' )|(misinfo_snopes['fact_rating_phase1'] == 'false' )]

#changing name of the columns 
misinfo_snopes= misinfo_snopes.rename(columns={'fact_rating_phase1':'Label','original_article_text_phase2':'Article'})
misinfo_snopes=misinfo_snopes.replace('true','True')
misinfo_snopes=misinfo_snopes.replace('false','False')

misinfo_snopes_original.head(1)

,fact_rating_phase1,snopes_url_phase1,article_title_phase1,article_category_phase1,article_date_phase1,article_claim_phase1,article_origin_url_phase1,index_paragraph_phase1,page_is_first_citation_phase1,error_phase2,original_article_text_phase2,article_title_phase2,publish_date_phase2,author_phase2,Jerry-label,Jill-label,Fatemeh-label,notes,original order,Agreement
0,mixture,https://www.snopes.com/fact-check/elizabeth-wa...,Did Elizabeth Warren Lie About Her Native Amer...,Politics Politicians,30-Oct-14,Elizabeth Warren lives in a multi-million-doll...,https://www.bostonglobe.com/metro/2012/01/13/e...,3,True,No Error,"Elizabeth Warren, the Harvard Law School profe...","Elizabeth Warrens salary topped $700,000 over ...",1/13/12,Noah Bierman,context,context,context,NaN,1,NaN


In [0]:
set (misinfo_snopes_original.fact_rating_phase1)

{'FALSE', 'TRUE', 'mixture', 'mostly false', 'mostly true'}

In [0]:
misinfo_asr_ft_etal_emergent.head(2)

,emergent_url_phase1,category_phase1,fact_tag_phase1,claim_phase1,claim_description_phase1,article_tags_phase1,article_date_phase1,article_tracking_body_phase1,original_url_phase1,error_phase2,original_article_text_phase2,article_title_phase2,publish_date_phase2,author_phase2
0,http://www.emergent.info/disney-collector-is-f...,Culture,Unverified,Claim: The woman behind the hugely popular DC ...,The channel has racked up more than 2.8 billio...,Daiane DeJesus&DC Toys Collector&Porm&Sandy Su...,Jul 3,The Daily Mail published a story on Feb 24 tha...,http://www.dailymail.co.uk/news/article-295824...,No Error,"The highest paid 'performer' on You Tube, who ...",Brazilian EX-porn star Diane DeJesus is figure...,2015-02-24,Paul Thompson In Orlando
1,http://www.emergent.info/tbs-speeds-up-Seinfeld,Culture,Unverified,Claim: TBS speeds up Seinfeld episodes to fit ...,"A video posted on YouTube, and then shared on ...",Seinfeld&TBS&TV,Jul 3,On Feb. 15 Reddit user Electrorocket posted a ...,http://www.avsforum.com/forum/34-hdtv-programm...,No Error,Quote: Aleron Ives The nomenclature seems to h...,TBS speeds up Seinfeld a full 7.5 percent - he...,NaN,Ash Sharma


In [0]:
#Parez_Rosas dataset 
import glob

#collecting list of directories

folder_location=["/content/drive/My Drive/Colab Notebooks/Perez-Rosas_etal_2018/fakeNewsDatasets/fakeNewsDataset/fake/*.txt",
                 "/content/drive/My Drive/Colab Notebooks/Perez-Rosas_etal_2018/fakeNewsDatasets/fakeNewsDataset/legit/*.txt",
                 "/content/drive/My Drive/Colab Notebooks/Perez-Rosas_etal_2018/fakeNewsDatasets/celebrityDataset/fake/*.txt",
                 "/content/drive/My Drive/Colab Notebooks/Perez-Rosas_etal_2018/fakeNewsDataset/celebrityDataset/legit/*.txt"]
#creating dataframe
Parez_Rosas= pd.DataFrame(columns=['Article','Label'])
for folder in folder_location:
    file_list = [f for f in glob.glob(folder)]
    for file in file_list:
        f=open(file, "r", encoding="utf8")
        content= f.read()
        if "legit" in file:
            Parez_Rosas=Parez_Rosas.append({'Article':content,'Label':'True'},ignore_index=True)
        else:
            Parez_Rosas=Parez_Rosas.append({'Article':content,'Label':'False'},ignore_index=True)

In [0]:
Parez_Rosas.shape
misinfo_snopes.shape

(312, 2)

In [0]:
label_frames = [misinfo_emergent, misinfo_snopes, Parez_Rosas]

labeled_dataset = pd.concat(label_frames,sort=False,ignore_index=True)
labeled_dataset=labeled_dataset.reset_index(drop=True)
# labeled_dataset.groupby('Label').count()

#saving in csv file 
labeled_dataset.to_csv('Labelled_Dataset.csv',index=False)
labeled_dataset.shape

(1092, 2)

In [0]:
#group 3 

#nela_gt = The final set of article data is arranged in an sqlite data, 
#with date, source, title, and cleaned text content for each article. 
#The labels are provided in CSV format, with rows being sources and columns being each label 
#gathered from all the assessment sites
nela_gt= pd.read_csv("/content/drive/My Drive/Colab Notebooks/group3/NELA-GT-2018_Norregaard_etal_2019/dataverse_files/labels.csv")

In [0]:
nela_gt_label= nela_gt.fillna(0)
nela_gt_label.head(5)

,Unnamed: 0,"NewsGuard, Does not repeatedly publish false content","NewsGuard, Gathers and presents information responsibly","NewsGuard, Regularly corrects or clarifies errors","NewsGuard, Handles the difference between news and opinion responsibly","NewsGuard, Avoids deceptive headlines","NewsGuard, Website discloses ownership and financing","NewsGuard, Clearly labels advertising","NewsGuard, Reveals who's in charge, including any possible conflicts of interest","NewsGuard, Provides information about content creators","NewsGuard, score","NewsGuard, overall_class","Pew Research Center, known_by_40%","Pew Research Center, total","Pew Research Center, consistently_liberal","Pew Research Center, mostly_liberal","Pew Research Center, mixed","Pew Research Center, mostly conservative","Pew Research Center, consistently conservative","Wikipedia, is_fake","Open Sources, reliable","Open Sources, fake","Open Sources, unreliable","Open Sources, bias","Open Sources, conspiracy","Open Sources, hate","Open Sources, junksci","Open Sources, rumor","Open Sources, blog","Open Sources, clickbait","Open Sources, political","Open Sources, satire","Open Sources, state","Media Bias / Fact Check, label","Media Bias / Fact Check, factual_reporting","Media Bias / Fact Check, extreme_left","Media Bias / Fact Check, right","Media Bias / Fact Check, extreme_right","Media Bias / Fact Check, propaganda","Media Bias / Fact Check, fake_news","Media Bias / Fact Check, some_fake_news","Media Bias / Fact Check, failed_fact_checks","Media Bias / Fact Check, conspiracy","Media Bias / Fact Check, pseudoscience","Media Bias / Fact Check, hate_group","Media Bias / Fact Check, anti_islam","Media Bias / Fact Check, nationalism","Allsides, bias_rating","Allsides, community_agree","Allsides, community_disagree","Allsides, community_label","BuzzFeed, leaning","PolitiFact, Pants on Fire!","PolitiFact, False","PolitiFact, Mostly False","PolitiFact, Half-True","PolitiFact, Mostly True","PolitiFact, True"
0,21stCenturyWire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,conspiracy_pseudoscience,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,left,0.0,0.0,0.0,0.0,0.0,0.0
1,ABC News,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,95.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_center_bias,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lean Left,8964.0,6949.0,somewhat agree,0,0.0,0.0,0.0,0.0,0.0,0.0
2,AMERICAblog News,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,left,0.0,0.0,0.0,0.0,0.0,0.0
3,Activist Post,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,conspiracy_pseudoscience,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,left,0.0,0.0,0.0,0.0,0.0,0.0
4,Addicting Info,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,left_bias,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,left,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
import sqlite3
conn = sqlite3.connect("/content/drive/My Drive/Colab Notebooks/group3/NELA-GT-2018_Norregaard_etal_2019/articles.db")
# con = conn.cursor()

# reading from Sqlite LIMIT is 100 , do increase the number while 
nelagt_df_original = pd.read_sql_query("SELECT * from articles LIMIT 2000", conn)
nelagt_df= pd.DataFrame(columns=['Article','Label'])
nelagt_df['Article'] =nelagt_df_original['content']
# nelagt_df =nelagt_df.rename(columns={'content':'Article'})
nelagt_df

,Article,Label
0,Since Donald Trump Jr. has admitted on Twitter...,NaN
1,The Republican-written and Republican-altered ...,NaN
2,The world is still laughing at the Trump admin...,NaN
3,Visiting the Matanuska banana plantation is no...,NaN
4,"In Croatia, thousands of people with mental il...",NaN
...,...,...
1995,"Despite this weeks stock market tumble, faith ...",NaN
1996,Interior Secretary Ryan Zinke is working to sa...,NaN
1997,White House press secretary Sarah Sanders had ...,NaN
1998,White House press secretary Sarah Sanders said...,NaN


In [0]:
raskin_etal_original = pd.read_csv("/content/drive/My Drive/Colab Notebooks/group3/newsfiles/newsfiles/fulltrain.csv",names=['Label','Article'],header=None)
# raskin_etal[1][1],raskin_etal[0][1]
raskin_etal = raskin_etal_original

#with no label
raskin_etal['Label']='NaN'
raskin_etal

,Label,Article
0,NaN,"A little less than a decade ago, hockey fans w..."
1,NaN,The writers of the HBO series The Sopranos too...
2,NaN,Despite claims from the TV news outlet to offe...
3,NaN,After receiving 'subpar' service and experienc...
4,NaN,After watching his beloved Seattle Mariners pr...
...,...,...
48849,NaN,The ruling Kuomintang (KMT) has claimed owners...
48850,NaN,The Taipei city government has encouraged the ...
48851,NaN,President Ma Ying-jeou said Friday that a park...
48852,NaN,The families of the four people who were kille...


In [0]:
Unlabel_frames = [raskin_etal,nelagt_df]

Unlabeled_dataset = pd.concat(Unlabel_frames,sort=False,ignore_index=True)
Unlabeled_dataset=Unlabeled_dataset.reset_index(drop=True)

#saving in csv file 
Unlabeled_dataset.to_csv('Unlabeled_dataset.csv',index=False)

#unlabelled dataset

Unlabeled_dataset.shape

(50854, 2)

In [0]:
labeled_dataset.head(1)

(1092, 2)

In [0]:
#Reading data 
path_labelled = "/content/drive/My Drive/Colab Notebooks/Labelled_Dataset.csv";



data = labeled_dataset

#removing missing rows 
missing_rows=[]
for i in range(len(data)):
  if data.loc[i, 'Article'] != data.loc[i, 'Article']:
    missing_rows.append(i)
data = data.drop(missing_rows).reset_index().drop(['index'],axis=1)
#cleaning data 
for i in range(len(data)):
        data.loc[i, 'Article'] = cleanup(data.loc[i,'Article'])

#count of labels in the dataset 
data.groupby('Label').count()
data['Article'][0]

'know you are troubl person blood org rever journalist person still anchor chair sheepishli admit had not travel helicopt hit enemi fire brokaw want person head platter org sourc said make lot nois org lesser journalist produc would immedi fire suspend fals report date person date acknowledg repeatedli said aboard chopper hit rocketpropel grenad report trip gpe actual safe travel differ aircraft brokaw cardin still date news anchor person came back gpe expedit insid said knew stori person later spout bunk person former org news presid person knew fals stori long time extrem uncomfort it sourc said org news exec counsel stop tell tale person still took anchor seat date news broadcast date time work rock day despit call dismiss did not address issu broadcast go suspend reprimand way full support org news necardinalrk sourc said mani person colleagu believ claim simpli conflat cardin version happen date meet day take serious org sourc said believ person apolog air speak himself admit time

In [0]:
data.groupby('Label').count()

,Article
Label,
FALSE,51
False,526
TRUE,65
True,254
mixture,72
mostly false,53
mostly true,71


In [0]:
data_unlabel = Unlabeled_dataset

#removing missing rows 
missing_rows=[]
for i in range(len(data_unlabel)):
  if data_unlabel.loc[i, 'Article'] != data_unlabel.loc[i, 'Article']:
    missing_rows.append(i)
data_unlabel = data_unlabel.drop(missing_rows).reset_index().drop(['index'],axis=1)
#cleaning data 
for i in range(len(data_unlabel)):
        data_unlabel.loc[i, 'Article'] = cleanup(data_unlabel.loc[i,'Article'])

#count of labels in the dataset 
data_unlabel.groupby('Label').count()
data_unlabel['Article'][0]